In [ ]:
import simpy 
import numpy as np
import random
import matplotlib.pyplot as plt

In [ ]:
"""
Bank renege example

Covers:

- Resources: Resource
- Condition events

Scenario:
  A counter with a random service time and customers who renege. Based on the
  program bank08.py from TheBank tutorial of SimPy 2. (KGM)

"""
import random

import simpy


RANDOM_SEED = 42
NEW_CUSTOMERS = 10000 # Total number of customers
LAMBD = 1/12
MU = 1/11

waiting_times = []
service_times = []

def source(env, number, lambd, mu, counter):
    """Source generates customers randomly"""
    t0 = np.random.exponential(1/lambd) # Arrival time of first customer
    yield env.timeout(t0)

    for i in range(number):
        c = customer(env, 'Customer%02d' % i, counter, mu)
        env.process(c)

        t = np.random.exponential(1/lambd)
        yield env.timeout(t)
   


def customer(env, name, counter, mu):
    """Customer arrives, is served and leaves."""
    arrive = env.now
    print('%7.4f %s: Here I am' % (arrive, name))

    with counter.request() as req: 
        yield req
        wait = env.now - arrive
        waiting_times.append(wait)
        print('%7.4f %s: Waited %6.3f' % (env.now, name, wait))
        service_time = np.random.exponential(1/mu)
        service_times.append(service_time)

        yield env.timeout(service_time)
        print('%7.4f %s: Finished' % (env.now, name))
    
    if len(waiting_times) == 5:
      print(waiting_times)
        

In [ ]:
random.seed(RANDOM_SEED)
def mmc_sim(new_customers, lambd, mu, capacity):
  waiting_times = []
  service_times = []
  env = simpy.Environment()

  # Start processes and run
  counter = simpy.Resource(env, capacity=capacity)
  env.process(source(env, new_customers, lambd, mu, counter))
  env.run()
  
  return waiting_times, service_times

In [ ]:
l_wt_mm1 = []
lambd_1 = 1/12
mu = 1/10


for i in range(101): 
    waiting_times, service_times = mmc_sim(100, 1/12, 1/11, 1)

    l_wt_mm1.append(waiting_times)

l_wt_mm1 = l_wt_mm1[:-1]

l_wt_mm2 = []
for i in range(101):
    waiting_times, service_times = mmc_sim(100, 2/12, 1/11, 1)

    l_wt_mm2.append(waiting_times)

l_wt_mm2 = l_wt_mm2[:-1]

mm1_means = np.mean(l_wt_mm1)
mm2_means = np.mean(l_wt_mm2)

mm1_std = np.std(l_wt_mm1)
mm2_std = np.std(l_wt_mm2)

In [ ]:
print(np.mean(mm1_means))
print(np.mean(mm2_means))

In [ ]:
sojourn_time = waiting_times + service_times
plt.hist(waiting_times, bins = 15)
plt.show()
plt.hist(service_times, bins = 15)
plt.show()
plt.hist(sojourn_time, bins = 15)
plt.show()

In [ ]:
rho = LAMBD/MU
print(f'Theoretical mean waiting time = {(rho/MU)/(1/rho)}')
print(f'Empirical mean waiting time = {np.mean(waiting_times)}')

In [ ]:
p_0_inverse = 0
rho = LAMBD/(c*MU)

for n in range(c):
    p_0_inverse += (c*rho)**n/math.factorial(n) + ((c*rho)**c/math.factorial(c))/(1-rho)

p_0 = 1/p_0_inverse
p_c = (c*rho)**c/math.factorial(c) * p_0
delay_probability = p_c/(1-rho)
E_W = delay_probability * (1/(1-rho)) * 1/(c*MU)

print(f'Theoretical mean waiting time = {E_W}')
print(f'Empirical mean waiting time = {np.mean(waiting_times)}')